In [1]:
!pip install pandas -q
!pip install denseweight

import numpy as np
import pandas as pd
from denseweight import DenseWeight


# File paths
train_file = "./extraextra_lin_unchangedinp_log1p_train_data.csv"
test_file = "./extraextra_lin_unchangedinp_log1p_test_data.csv"

# Load the datasets
train_df = pd.read_csv(train_file)
test_df = pd.read_csv(test_file)

dw = DenseWeight(alpha=0.75)
print(pd.concat([train_df["Label"], test_df["Label"]]).to_numpy().shape)
weights = dw.fit(pd.concat([train_df["Label"], test_df["Label"]]).to_numpy())
train_weights = dw(train_df["Label"].to_numpy())
test_weights = dw(test_df["Label"].to_numpy())

print(dw([0.01]))
print(dw([0.1]))
print(dw([1]))
print(dw([6]))
print(dw([200]))
print(dw([400]))

# Display the first few rows of each dataset
print("Train Data:")
print(train_df.info())

print("\nTest Data:")
print(test_df.info())



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


(1000000,)
[0.43140687]
[1.44147439]
[1.69091653]
[1.70841557]
[1.70968533]
[1.70968533]
Train Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Series1  800000 non-null  float64
 1   Series2  800000 non-null  float64
 2   Series3  800000 non-null  float64
 3   Label    800000 non-null  float64
dtypes: float64(4)
memory usage: 24.4 MB
None

Test Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   Series1  200000 non-null  float64
 1   Series2  200000 non-null  float64
 2   Series3  200000 non-null  float64
 3   Label    200000 non-null  float64
dtypes: float64(4)
memory usage: 6.1 MB
None


In [2]:
train_df.describe()

,Series1,Series2,Series3,Label
count,800000.000000,800000.000000,800000.000000,8.000000e+05
mean,-0.002159,0.000114,0.500670,2.498894e-01
std,2.632232,0.583146,0.291228,5.131788e-01
min,-6.000000,-1.000000,0.001000,0.000000e+00
25%,-1.346939,-0.494949,0.253273,4.289651e-09
50%,-0.002121,-0.010101,0.505545,4.391191e-02
75%,1.346939,0.515152,0.757818,3.308511e-01
max,6.000000,1.000000,1.000000,5.991320e+00


In [3]:
# install norse and pytorch
!pip install sinabs


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch

class DataFrameDataset(Dataset):
    def __init__(self, dataframe):
        self.inputval = dataframe["Series1"]
        self.mean = dataframe["Series2"]
        self.std = dataframe["Series3"]
        self.labels = dataframe["Label"]

    def __len__(self):
        return len(self.inputval)

    def __getitem__(self, idx):
        inputval = self.inputval.iloc[idx]
        mean = self.mean.iloc[idx]
        std = self.std.iloc[idx]
        labels = self.labels.iloc[idx]
        return inputval, mean, std, labels
    
# Create the datasets
train_dataset = DataFrameDataset(train_df)
test_dataset = DataFrameDataset(test_df)


In [5]:
from torch.utils.data import WeightedRandomSampler

train_sampler = WeightedRandomSampler(weights=train_weights, num_samples=len(train_weights))


In [6]:
# Encode input data
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True) # , sampler=train_sampler)
test_loader = DataLoader(test_dataset, batch_size=64) #, shuffle=True)


In [7]:
import torch.nn as nn
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [8]:
import torch

TRAINING_PREFIX = "extraextra_lin_256_2HL_log1p"

ann = nn.Sequential(
    nn.Linear(3, 256),  # Input layer: 3 features (mu, sigma, x)
    nn.ReLU(),
    nn.Linear(256, 256),  # first hidden layer
    nn.ReLU(),
    nn.Linear(256, 256),  # second hidden layer
    nn.ReLU(),
    nn.Linear(256, 1)    # Output layer: single value for f(x; mu, sigma)
)

# Load checkpoint
#ann.load_state_dict(torch.load(f"results/{TRAINING_PREFIX}/ann_epoch_300.pth"))

ann.to(device)

total_params = sum(p.numel() for p in ann.parameters() if p.requires_grad)
print(f"Total Parameters: {total_params}")


Total Parameters: 132865


In [9]:
from sinabs.from_torch import from_model

num_time_steps_per_sample = 100

sinabs_model = from_model(ann, input_shape=(3,), add_spiking_output=True, synops=False, num_timesteps=num_time_steps_per_sample)
sinabs_model

Network(
  (spiking_model): Sequential(
    (0): Linear(in_features=3, out_features=256, bias=True)
    (1): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
    (4): Linear(in_features=256, out_features=256, bias=True)
    (5): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
    (6): Linear(in_features=256, out_features=1, bias=True)
    (spike_output): IAFSqueeze(spike_threshold=Parameter containing:
    tensor(1.), min_v_mem=Parameter containing:
    tensor(-1.), batch_size=-1, num_timesteps=100)
  )
  (analog_model): Sequential(
    (0): Linear(in_features=3, out_features

In [10]:
!pip install torcheval -q

from IPython.display import clear_output
from matplotlib import pyplot as plt
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display
from torcheval.metrics import R2Score


def create_loss_plot():
    fig = go.FigureWidget()
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Train Loss', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines', name='Eval Loss', line=dict(color='orange')))

    # Configure layout
    fig.update_layout(title='Training and Evaluation Losses',
                    xaxis_title='Epoch',
                    yaxis_title='Loss',
                    template='plotly_dark')

    # Display the figure widget
    display(fig)
    return fig

def update_loss_plot(fig, train_loss, eval_loss, from_epoch=0):
    if from_epoch != 0:
        with fig.batch_update():
            fig.data[0].x = list(range(from_epoch, len(train_loss)))
            fig.data[0].y = train_loss[from_epoch:]
            fig.data[1].x = list(range(from_epoch, len(eval_loss)))
            fig.data[1].y = eval_loss[from_epoch:]
    elif len(train_loss) < 30:
        with fig.batch_update():
            fig.data[0].x = list(range(len(train_loss)))
            fig.data[0].y = train_loss
            fig.data[1].x = list(range(len(eval_loss)))
            fig.data[1].y = eval_loss
    else:
        with fig.batch_update():
            fig.data[0].x = list(range(len(train_loss) - 30, len(train_loss)))
            fig.data[0].y = train_loss[-30:]
            fig.data[1].x = list(range(len(eval_loss) - 30, len(eval_loss)))
            fig.data[1].y = eval_loss[-30:]        
    

import numpy as np
def gaussian_probability(x, y, z):
    return (1 / (np.sqrt(2 * np.pi) * z)) * np.exp(-((x - y) ** 2) / (2 * z ** 2))

from tqdm.auto import tqdm
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

def do_epoch(loader, model, optimizer, device, n_epochs: int, current_epoch: int, train: bool = False, scheduler=None):
    if train:
        model.train()
    else:
        model.eval()
    epoch_loss = 0
    metric = R2Score(device=device)
    for inputvals, means, stds, labels in tqdm(loader, desc=f"{'Epoch' if train else 'Eval Epoch'} {current_epoch+1}/{n_epochs}"):
        inputs = torch.stack((inputvals, means, stds), dim=1).float()
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        metric.update(outputs.squeeze(), labels)
        if train:
            optimizer.zero_grad()
            loss = F.huber_loss(outputs.squeeze(), labels.float())
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
        else:
            loss = F.huber_loss(outputs.squeeze(), labels.float())
        epoch_loss += loss.item()
    
    # For Plateau scheduler
    if scheduler is not None and not train:
        print("sched step")
        scheduler.step(epoch_loss)
    return epoch_loss, metric.compute()


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# Scheduler
!pip install pytorch_warmup

from pytorch_warmup import LinearWarmup

# Train

ann.train()

N_EPOCHS = 1000
LR = 0.0004
SAVE_EVERY = 50

# Create subfolder for this loop
import os
import shutil

if os.path.exists(f"./results/{TRAINING_PREFIX}"):
    shutil.rmtree(f"./results/{TRAINING_PREFIX}")
os.makedirs(f"./results/{TRAINING_PREFIX}")

optim = torch.optim.AdamW(ann.parameters(), lr=LR)
#optim = torch.optim.SGD(ann.parameters(), lr=LR)
#opt_step = torch.compile(optim.step, mode="reduce-overhead")

num_steps = len(train_loader) * N_EPOCHS

lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, mode='min', factor=0.25, patience=10, verbose=True)
#    optim,
#    milestones=[i * len(train_loader) for i in [250, 500]])
#warmup_scheduler = LinearWarmup(optim, warmup_period=(len(train_loader) * 50) )


last_loss = 0
last_eval_loss = 0

epoch_loss_list = []
eval_loss_list = [] 

fig = create_loss_plot()

for epoch in range(N_EPOCHS):
    if (epoch + 1) % 30 == 0:
        clear_output(wait=True)
        fig = create_loss_plot()
        update_loss_plot(fig, epoch_loss_list, eval_loss_list)
    
    epoch_loss, train_R2_score = do_epoch(train_loader, ann, optim, device, n_epochs=N_EPOCHS, current_epoch=epoch, train=True, scheduler=lr_scheduler)
    eval_epoch_loss, eval_R2_score = do_epoch(test_loader, ann, None, device, n_epochs=N_EPOCHS, current_epoch=epoch, train=False, scheduler=lr_scheduler)
    eval_loss_list.append(eval_epoch_loss/len(test_loader))
    print(f"Epoch {epoch+1}, LR: {lr_scheduler.get_last_lr()}, Train Loss: {epoch_loss/len(train_loader)}, Diff: {epoch_loss/len(train_loader) - last_loss}, Eval Loss: {eval_epoch_loss/len(test_loader)}, Diff Eval: {eval_epoch_loss/len(test_loader) - last_eval_loss}, Train R2 Score: {train_R2_score}, Eval R2 Score: {eval_R2_score}")
    last_loss = epoch_loss/len(train_loader)
    last_eval_loss = eval_epoch_loss/len(test_loader)
    epoch_loss_list.append(epoch_loss/len(train_loader))
    update_loss_plot(fig, epoch_loss_list, eval_loss_list)
    if epoch % SAVE_EVERY == 0:
        torch.save(ann.state_dict(), f"./results/{TRAINING_PREFIX}/ann_epoch_{epoch}.pth")
    elif epoch == N_EPOCHS - 1:
        torch.save(ann.state_dict(), f"./results/{TRAINING_PREFIX}/ann_epoch_{epoch}.pth") 

FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'mode': 'lines',
              'name': 'Train Loss',
              'type': 'scatter',
              'uid': '1c03b424-b90e-40af-acbc-aaba87699e19',
              'x': [],
              'y': []},
             {'line': {'color': 'orange'},
              'mode': 'lines',
              'name': 'Eval Loss',
              'type': 'scatter',
              'uid': 'ae89a786-d2f7-4284-925d-9a0754e078a9',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Training and Evaluation Losses'},
               'xaxis': {'title': {'text': 'Epoch'}},
               'yaxis': {'title': {'text': 'Loss'}}}
})

Epoch 630/1000:   0%|          | 0/12500 [00:00<?, ?it/s]

Eval Epoch 630/1000:   0%|          | 0/3125 [00:00<?, ?it/s]

sched step
Epoch 630, LR: [6.103515625e-09], Train Loss: 2.479790812515148e-06, Diff: -4.7079287583055e-10, Eval Loss: 5.5496209709872345e-06, Diff Eval: -2.214549294876629e-09, Train R2 Score: 0.999981164932251, Eval R2 Score: 0.9999580383300781


Epoch 631/1000:   0%|          | 0/12500 [00:00<?, ?it/s]

...

Epoch 655/1000:   0%|          | 0/12500 [00:00<?, ?it/s]

Eval Epoch 655/1000:   0%|          | 0/3125 [00:00<?, ?it/s]

sched step
Epoch 655, LR: [6.103515625e-09], Train Loss: 2.480250081775921e-06, Diff: -1.4648015326202593e-09, Eval Loss: 5.54937692173553e-06, Diff Eval: 1.6933006190813103e-09, Train R2 Score: 0.999981164932251, Eval R2 Score: 0.9999580383300781


Epoch 656/1000:   0%|          | 0/12500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
#torch.save(ann.state_dict(), "largerer_ann_new_data5.pth")


In [12]:
fig = create_loss_plot()
update_loss_plot(fig, epoch_loss_list, eval_loss_list, 10)

FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'mode': 'lines',
              'name': 'Train Loss',
              'type': 'scatter',
              'uid': '6c0504d7-16bb-41b8-975d-82ba5e5f2547',
              'x': [],
              'y': []},
             {'line': {'color': 'orange'},
              'mode': 'lines',
              'name': 'Eval Loss',
              'type': 'scatter',
              'uid': '482614b7-0ffe-474c-bc86-1962277ccb6d',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Training and Evaluation Losses'},
               'xaxis': {'title': {'text': 'Epoch'}},
               'yaxis': {'title': {'text': 'Loss'}}}
})

In [13]:
epoch_loss_list

[0.010720561613654717,
 0.005650727769082005,
 0.004973712196938722,
 0.0046965876476417175,
 0.00449116287927187,
 0.0044736452529025705,
 0.00434148402540508,
 0.004304189840590843,
 0.0043479884606443375,
 0.004332168479999127,
 0.004299884850940253,
 0.004254439220912136,
 0.00421730754099568,
 0.004236685128641839,
 0.004213748104372171,
 0.004182607872355438,
 0.004251838786280278,
 0.004290557541898742,
 0.0042712727683148115,
 0.004242378014136594,
 0.004250861869595573,
 0.00425550736627858,
 0.004341588957470994,
 0.004267976844588484,
 0.004295712128373216,
 0.004355830985663124,
 0.004358745869377599,
 0.004367734752605648,
 0.00434205544977729,
 0.004377770489534632,
 0.004356311271676313,
 0.004363446383026967,
 0.004342417285856391,
 0.004399272141062902,
 0.0034898571438213275,
 0.0034030793314072253,
 0.0033154675950014824,
 0.0032616962320990662,
 0.0032274479818930014,
 0.0031887777890863343,
 0.0032312808225748086,
 0.003186244974969286,
 0.003142077289710269,
 0.00

In [14]:
ann.eval()
transformed = [[0.5, 0.5, 0.5]]
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())


np.float64(0.7985268002609114)

In [15]:
gaussian_probability(0.5, 0.5, 0.5)

np.float64(0.7978845608028654)

In [16]:
transformed = [[0.1, 0.1, 0.1]]
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

np.float64(3.9791190668689422)

In [17]:
gaussian_probability(0.1, 0.1, 0.1)

np.float64(3.989422804014327)

In [18]:
transformed = [[0.01, 0.01, 0.01]]
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

np.float64(41.86783290284835)

In [19]:
gaussian_probability(0.01, 0.01, 0.01)

np.float64(39.894228040143275)

In [20]:
transformed = [[0.9, 0.9, 0.9]]
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

np.float64(0.4408715498811401)

In [21]:
gaussian_probability(0.9, 0.9, 0.9)

np.float64(0.44326920044603635)

In [22]:
transformed = [[0.9, 0.7, 0.1]]
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

np.float64(0.5431544288899663)

In [23]:
gaussian_probability(0.9, 0.7, 0.1)

np.float64(0.53990966513188)

In [24]:
transformed = [[0.643, 0.7, 0.1]]
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

np.float64(3.3789679900971876)

In [25]:
gaussian_probability(0.643, 0.7, 0.1)

np.float64(3.3912431320419234)

In [26]:
transformed = [[4, 0.3, 0.2]]
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

np.float64(2.908748968321537e-05)

In [27]:
gaussian_probability(4, 0.3, 0.2)

np.float64(9.57716245835995e-75)

In [28]:
train_dataset.mean.unique()

array([ 0.13131313, -0.23232323, -0.51515152,  0.43434343,  0.81818182,
       -0.49494949, -0.81818182,  0.05050505,  0.15151515,  0.45454545,
        0.35353535, -0.33333333,  0.09090909,  0.71717172, -0.97979798,
        0.6969697 , -0.95959596, -0.83838384,  0.39393939,  0.7979798 ,
        0.27272727, -0.43434343, -0.71717172, -0.93939394, -0.53535354,
        0.75757576, -0.17171717, -0.39393939, -0.37373737, -0.45454545,
        0.37373737, -0.21212121,  0.19191919, -0.09090909, -0.61616162,
        0.53535354,  0.87878788, -0.87878788,  0.47474747, -0.85858586,
       -0.13131313, -0.67676768, -0.03030303, -0.11111111,  0.73737374,
       -0.63636364, -0.01010101, -0.7979798 ,  0.65656566,  0.95959596,
       -0.73737374,  1.        , -0.5959596 ,  0.29292929,  0.49494949,
        0.17171717,  0.25252525,  0.01010101, -0.29292929,  0.03030303,
        0.8989899 ,  0.55555556,  0.67676768, -0.77777778, -0.31313131,
        0.33333333,  0.51515152,  0.77777778,  0.41414141,  0.07

In [29]:
train_dataset.std.unique()

array([0.68718182, 0.23309091, 0.15236364, 0.92936364, 0.57618182,
       0.97981818, 0.001     , 0.93945455, 0.04136364, 0.03127273,
       0.98990909, 0.54590909, 0.09181818, 0.42481818, 0.56609091,
       0.34409091, 0.14227273, 0.26336364, 0.01109091, 0.556     ,
       0.71745455, 0.81836364, 0.59636364, 0.45509091, 0.47527273,
       0.112     , 0.60645455, 0.61654545, 0.43490909, 0.69727273,
       0.75781818, 0.46518182, 0.82845455, 0.32390909, 0.16245455,
       0.91927273, 0.18263636, 0.29363636, 0.63672727, 0.76790909,
       0.96972727, 0.58627273, 0.74772727, 0.86881818, 0.36427273,
       0.90918182, 0.889     , 0.12209091, 0.27345455, 0.73763636,
       0.62663636, 0.65690909, 0.49545455, 0.87890909, 0.10190909,
       0.778     , 0.89909091, 0.17254545, 0.20281818, 0.48536364,
       0.08172727, 0.94954545, 0.24318182, 0.50554545, 0.41472727,
       0.667     , 0.31381818, 0.39454545, 0.25327273, 0.37436364,
       0.05145455, 0.51563636, 0.21290909, 0.334     , 1.     

In [30]:
# std 1.25 and mean -3.04
transformed = [[-2, -3.04, 1.25]]
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

np.float64(0.20334854540768302)

In [31]:
gaussian_probability(-2, -3.04, 1.25)

np.float64(0.22578002723581)

In [32]:
transformed = [[0.999, 1, 0.001]]
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

np.float64(242.76307435675864)

In [33]:
gaussian_probability(0.999, 1, 0.001)

np.float64(241.97072451914318)

In [34]:
transformed = [[1, 1, 0.001]]
tensor = torch.tensor(transformed).to(device).float()
np.expm1(ann(tensor).item())

np.float64(397.1416407205497)

In [35]:
gaussian_probability(1, 1, 0.001)

np.float64(398.94228040143275)